## HW4 - Dynamic Programming and Uncertainty

In this final homework assignment you will be asked to solve 2 problems using dynamic programming. In the first, we will focus on a completely static offline problem. In the second, we will use both dynamic programming and some of the ideas we learned about in our 1-lecture simulation unit.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

#### Problem 1: Checkerboard

Consider the following problem. You are at the bottom-left corner of a chessboard ("A1"). Each field on the board has a value. The value of a field is stored in the dictionary "values" defined in the cell below. For each field you visit, you collect the value of that field; hwowever, you are only allowed to move "up" or "right" (e.g., from "A1" to "B1" or from "A1" to "A2"). Find the maximum value you can collect as you move from "A1" eventually to "H8"? What sequence of fields to visit will yield that value?

(An optimal solution should yield 1111 in value)

In [2]:
values = {('A', 1): 87, ('A', 2): 6, ('A', 3): 81, ('A', 4): 89, ('A', 5): 99, ('A', 6): 8, ('A', 7): 47,
 ('A', 8): 81, ('B', 1): 18, ('B', 2): 17, ('B', 3): 50, ('B', 4): 66, ('B', 5): 91, ('B', 6): 47, ('B', 7): 98,
 ('B', 8): 55, ('C', 1): 21, ('C', 2): 1, ('C', 3): 16, ('C', 4): 9, ('C', 5): 70, ('C', 6): 96, ('C', 7): 93,
 ('C', 8): 34, ('D', 1): 19, ('D', 2): 63, ('D', 3): 52, ('D', 4): 74, ('D', 5): 50, ('D', 6): 5, ('D', 7): 90,
 ('D', 8): 45, ('E', 1): 35, ('E', 2): 3, ('E', 3): 30, ('E', 4): 43, ('E', 5): 62, ('E', 6): 57, ('E', 7): 20,
 ('E', 8): 54, ('F', 1): 51, ('F', 2): 70, ('F', 3): 70, ('F', 4): 49, ('F', 5): 20, ('F', 6): 8, ('F', 7): 5,
 ('F', 8): 92, ('G', 1): 70, ('G', 2): 79, ('G', 3): 21, ('G', 4): 86, ('G', 5): 42, ('G', 6): 76, ('G', 7): 67,
 ('G', 8): 33, ('H', 1): 1, ('H', 2): 26, ('H', 3): 78, ('H', 4): 53, ('H', 5): 39, ('H', 6): 73, ('H', 7): 24,
 ('H', 8): 85}

Hint: When at field "H8" you get a value of 85, but will not receive any more from moving. At field "H7" you receive 24, and you will have to move to "H8" next getting you another 85. At "G7" you get 67, and you get the better of moving to "G8" (33) and then to "H8" or to "H7" (24) and then to "H8". How much value will you collect in the remaining path starting from "G7"?

In [5]:
# Gathering letter and number combo's
letters = []
numbers = []
for square in values:
    if square[0] not in letters:
        letters.append(square[0])
    if square[1] not in numbers:
        numbers.append(square[1])
letters.sort()
numbers.sort()
        
        
# Creating blank copy to map the optimal values
opt_val = {}
for char in letters:
    for num in numbers:
        opt_val[char,num] = 0

# filling right most column and bottom most row
previous_val = 0
for char in reversed(letters):
    opt_val[(char,numbers[-1])] = previous_val + values[(char,numbers[-1])]
    previous_val = opt_val[(char,numbers[-1])]
    
previous_val = 0
for num in reversed(numbers):
    opt_val[(letters[-1],num)] = previous_val + values[(letters[-1],num)]
    previous_val = opt_val[(letters[-1],num)]
    
# Filling rest of opt_val matrix
for char in range(len(letters)-1,-1,-1):
    for num in range(len(numbers)-1,-1,-1):
        if opt_val[(letters[char],numbers[num])] == 0:
            opt_val[(letters[char],numbers[num])] = values[(letters[char],numbers[num])] + max(opt_val[(letters[char+1],numbers[num])],opt_val[(letters[char],numbers[num+1])])
print('optimal answer =', opt_val[(letters[0],numbers[0])])


opt_path = []
# Find next optimal path
def next_optimal_pos(char,num):
    pos = (letters[char],numbers[num])
    opt_path.append(pos)
    if pos == (letters[-1],numbers[-1]):
        print('Optimal Path =',opt_path)
        return
    
    if char == (len(letters)-1):
        return next_optimal_pos(char,num+1)
    if num == (len(numbers)-1):
        return next_optimal_pos(char+1,num)
    
    opt_1 = (letters[char+1],numbers[num])
    opt_2 = (letters[char],numbers[num+1])
    if opt_val[opt_1] > opt_val[opt_2]:
        return next_optimal_pos(char+1,num)
    else:
        return next_optimal_pos(char,num+1)

# Finding optimal path
next_optimal_pos(0,0)

optimal answer = 1111
Optimal Path = [('A', 1), ('A', 2), ('A', 3), ('A', 4), ('A', 5), ('B', 5), ('C', 5), ('C', 6), ('C', 7), ('D', 7), ('D', 8), ('E', 8), ('F', 8), ('G', 8), ('H', 8)]


#### Problem 2: Secretary problem (implement solution from class)

In class we saw the secretary problem, in which the CEO's goal was to maximize the expected score of the candidate. We saw the intuition then of what a solution should look like: she should accept in round $t$ if (in expectation) the remaining $t-1$ candidates will not be better. Use dynamic programming and simulation to identify the threshold for accepting with $t$ candidates remaining for $t=1,2,3,4,5$.

Hint: the cell below creates 100,000 samples of 5 random secretary arrivals. Use those to estimate (for a given threshold) the expected quality of the last arrival. You'd want to accept the second-to-last if he is better than that expected quality. You can also use these arrivals to estimate the probability that the second-to-last arrival would be better than the expected quality, and use those 2 together to determine the expected quality the CEO will get out of the last 2 combined, which you can then use for the third-to-last (and so forth)...

In [5]:
arrivals = {}
for k in range(10**5):
    arrivals[k] = np.random.randint(0,101,5)

In [6]:
thresholds = {}
for num in range(len(arrivals[0])-1,0,-1):
    exp_score = 0
    count = 0
    if num == len(arrivals[0])-1:
        for trial in arrivals.values():
            for score in trial:
                exp_score += score
                count += 1
        exp_score = exp_score/count
        thresholds[num] = exp_score
    else:
        for trial in arrivals.values():
            for score in trial:
                if score > thresholds[num+1]:
                    exp_score += score
                    count += 1
        exp_score = exp_score/count
        prob = (100 - thresholds[num+1])/100
        thresholds[num] = prob*exp_score + (1-prob)*thresholds[num+1]

for num in range(1,len(arrivals[0])):
        print('Round', num, 'threshold =', thresholds[num])

Round 1 threshold = 74.36928269349737
Round 2 threshold = 69.75417409009842
Round 3 threshold = 62.776451083413264
Round 4 threshold = 50.028144


Next, repeat the above, but rather than assuming that the quality score of each candidate is uniform between 0 and 100, assume it is normally distributed with mean 500 and standard deviation 20 (if you did the above right, this should only require a change in one line).

In [7]:
from scipy.stats import norm

arrivals = {}
for k in range(10**5):
    arrivals[k] = np.random.normal(500,20,5)

In [8]:
thresholds = {}
for num in range(len(arrivals[0])-1,0,-1):
    exp_score = 0
    count = 0
    if num == len(arrivals[0])-1:
        for trial in arrivals.values():
            for score in trial:
                exp_score += score
                count += 1
        exp_score = exp_score/count
        thresholds[num] = exp_score
    else:
        for trial in arrivals.values():
            for score in trial:
                if score > thresholds[num+1]:
                    exp_score += score
                    count += 1
        exp_score = exp_score/count
        prob = norm.cdf((thresholds[num+1]-500)/20)
        thresholds[num] = prob*exp_score + (1-prob)*thresholds[num+1]

for num in range(1,len(arrivals[0])):
        print('Round', num, 'threshold =', thresholds[num])

Round 1 threshold = 525.5697352197176
Round 2 threshold = 516.6791833348419
Round 3 threshold = 507.92806369445145
Round 4 threshold = 499.9582886129888
